This cell is responsible for managing imports.

In [1]:
from qiskit import Aer          # for simulator backend
from qiskit_machine_learning.algorithms import QSVC # quantum support vector classifier class
from qiskit_machine_learning.kernels.quantum_kernel import QuantumKernel # wraps feature map and backend combination to give to QSVC
import qiskit.circuit.library   # for feature maps
import numpy as np
import joblib                   # for persistence, caching, and parallelism

# for data sets and data set processing
import sklearn.datasets
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from qiskit_machine_learning.datasets.dataset_helper import features_and_labels_transform

Start by configuring joblib's caching and parallelism and the quantum backend. Parallelism and caching help save time and recover from power outages and shouldn't affect the final results. If the backend configuration is changed, the cache folder should be deleted so that the experiments run with the new backend and don't simply recall results from a previous backend.

In [2]:
# configure caching
cache_directory = "./joblib_cache"
memory = joblib.Memory(location=cache_directory, compress=False, mmap_mode=None)
# configure parallelism
worker_count = 6
parallel = joblib.Parallel(n_jobs=worker_count, backend="loky", batch_size=1, mmap_mode=None)
#configure backend
backend = Aer.get_backend("aer_simulator_statevector") # should configure this to mimic IBMQ backend
#backend.set_options(device='GPU')                      # enable GPU acceleration (comment this line to disable)

The functions in this cell are responsible for loading, preparing, and splitting data sets for input to the QSVM classifier.

In [3]:
# TODO: change this to allow specifying which classes to extract for binary classification,
# rather than simply extracting 2 arbitrary classes. If 2 arbitrary classes are specified anyway,
# at least make them manually selected and identifiable rather than seemingly random.
def extract_binary_classes(feature_array, label_array):
    """Takes a numpy array of feature vectors and a numpy array of labels
    and returns transformed numpy arrays with the number of classes reduced
    to 2."""
    classes = list(set(label_array))[:2] # get the first 2 unique labels as classes
    class_map = {classes[0]:0, classes[1]:1} # convert labels to 0 and 1 (needed for training step)
    # construct a feature and label description with information from only the first 2 classes
    features = []
    labels = []
    for (feature, label) in zip(feature_array, label_array):
        if label in classes:
            features.append(feature)
            labels.append(label)
    return (np.array(features), np.array(labels))

def process_dataset(dataset, qubit_count=4, binary_classification=True):
    """Performs scaling and dimensionality reduction on all feature vectors of a data set,
    then returns the processed vectors. It will also extract 2 classes from the data set
    if binary_classification is True, rather than leaving the data set as a multi-class data set.
    Some of this code is modified from the qiskit_machine_learning data set loading
    source code (check qiskit_machine_learning.datasets.digits source code for exact location
    of what was modified from)."""
    feature_vectors = dataset.data
    labels = dataset.target

    # maybe extract classes for binary classification
    if binary_classification:
        feature_vectors, labels = extract_binary_classes(feature_vectors, labels)

    # Now we standardize for gaussian around 0 with unit variance
    scaler = StandardScaler()
    scaler.fit(feature_vectors)
    feature_vectors = scaler.transform(feature_vectors)

    # Now reduce number of features to number of qubits
    pca = PCA(n_components=qubit_count)
    pca.fit(feature_vectors)
    feature_vectors = pca.transform(feature_vectors)

    # Scale to the range (-1,+1)
    minmax_scaler = MinMaxScaler((-1, 1)).fit(feature_vectors)
    feature_vectors = minmax_scaler.transform(feature_vectors)

    # perform some other transformation on the feature and label vectors
    # as was done in the qiskit_machine_learning source code
    dataset_dict = {label:np.array([feature_vector for feature_vector, feature_vector_label in zip(feature_vectors, labels)
                                    if feature_vector_label == label])
                    for label in list(set(labels))}
    feature_vectors, labels = features_and_labels_transform(dataset_dict, labels, one_hot=False)

    return feature_vectors, labels

def cross_fold_sets(data, labels, k=5, seed=22):
    "Given a data set's feature array, yield training and testing feature arrays for k-fold validation. If the same seed is used then the same subsets should be returned across different calls."
    kf = KFold(n_splits=k, shuffle=True, random_state=seed)
    # for each of the k train-test splits:
    for train_indices, test_indices in kf.split(data, labels):
        # helper function
        extract_elements = lambda array, indices: np.array([array[i] for i in indices])
        # get training and testing feature vectors
        train_features = extract_elements(data, train_indices)
        test_features = extract_elements(data, test_indices)
        # get training and testing labels
        train_labels = extract_elements(labels, train_indices)
        test_labels = extract_elements(labels, test_indices)
        # return current split values
        yield (train_features, train_labels, test_features, test_labels)

This cell defines a function that can be given some parameters determining a classifier, like the feature map to use, the data to train on, and the backend to run the training on.

In [4]:
# MAYBE DO: make batch size a parameter
def make_classifier(feature_map_instance, training_features, training_labels):
    """Given a feature map instance, training features and labels,
    creates, trains, and returns a QSVM classifier."""
    # Create a quantum kernel from the feature map and
    # backend to give to the QSVC class.
    batch_size = 1000           # this is the QuantumKernel default
    quantum_kernel = QuantumKernel(feature_map=feature_map_instance, batch_size=batch_size, quantum_instance=backend)
    # Create a QSVC instance
    qsvc = QSVC(quantum_kernel=quantum_kernel)
    # Perform training
    qsvc.fit(training_features, training_labels)
    # return classifier instance
    return qsvc

This cell is similar to the above cell in that it in effect takes a specification for a classifier, but the function instead returns the generalisation metrics of the classifier that is described.

In [5]:
# TODO: finalize what generalisation metrics should be used and calculate them (maybe also look at margin size)
# MAYBE DO: put parameters like feature count and number of repetitions in
# the argument list to make them independent variables of the experiments rather
# than constants.
# This class list is defined so that the feature map class argument to process_combination can be a number
# instead of a class instance. This allows caching to succeed.
feature_map_class_list = [qiskit.circuit.library.PauliFeatureMap,
                           qiskit.circuit.library.ZFeatureMap,
                           qiskit.circuit.library.ZZFeatureMap]
# This function is where most of the execution time
# lies since the model is also trained during its run time, so there
# is a large benefit to caching it to avoid recomputation after
# power failures. It is not a pure function however as it depends indirectly on the global
# values of feature_map_class_list and the backend used to compute the results, so
# the cache should be cleared if the backend or feature map list is changed.
# NOTE: caching doesn't work properly within a python notebook since namespaces are changed,
# so the notebook should be exported to a .py file then run if the ability to resume from
# interruptions is important.
@memory.cache
def process_combination(feature_map_class_number, data_split_tuple, repetitions, qubit_count=4):
    """Takes a feature map class number, dataset loading function, and a backend, and
    returns the generalisation metrics of the combination of arguments."""
    # Create the feature map instance.
    feature_map_instance = feature_map_class_list[feature_map_class_number](feature_dimension=qubit_count, reps=repetitions)
    # unpack the data split for binary classification
    train_features, train_labels, test_features, test_labels = data_split_tuple

    # create the classifier
    qsvc = make_classifier(feature_map_instance, train_features, train_labels)

    # get the classification accuracy on training and testing data as generalisation metrics
    train_accuracy = qsvc.score(train_features, train_labels)
    test_accuracy = qsvc.score(test_features, test_labels)
    # return the generalisation metrics and the trained model
    return train_accuracy, test_accuracy, qsvc

This cell defines a function that collects the generalisation information of all tested classifier configurations.

In [6]:
# This function takes a long time to run, so it benefits from parallelism.
# It is easily parallelised by running each data set's combinations with
# different workers.
# MAYBE DO: separate out the classifier configurations / combinations generation into
# a different function (for clarity). It could be called "generate_all_combinations"
# or something similar
def process_all_combinations(do_binary_classification=True):
    """Runs all experiments."""
    np.random.seed(22)     # set a fixed seed so that caching works
    # get a list of datasets loaded into memory
    datasets = [sklearn.datasets.load_breast_cancer(),
                sklearn.datasets.load_digits(),
                sklearn.datasets.load_iris(),
                sklearn.datasets.load_wine()]
    # corresponding human-readable names for recording results
    dataset_names = ["cancer", "digits", "iris", "wine"]

    # get a list of feature maps
    feature_map_class_numbers = range(len(feature_map_class_list)) # feature_map_class_list is a global list defined in the previous cell
    # corresponding human-readable names for recording results
    feature_map_names = ["Pauli", "Z", "ZZ"]

    # Number of qubits to use / number of features to reduce to.
    qubit_count = 4

    # Define choice of k for k-fold cross-validation. This
    # number determines how many equally sized disjoint
    # subsets to split the dataset into, after which each
    # is used as the testing set in turn with the remaining
    # subsets being used as the training set.
    cross_validation_splits = 5 # a value of 5 gives a 20:80 test:train split, and 5 total validation splits
    
    # Do some output to the user to give them a sense of how long running the experiments will take
    number_of_investigated_repetition_values = 4 # for trying depth = 2, 3, 4, and 5
    combination_count = len(datasets) * cross_validation_splits * (number_of_investigated_repetition_values + len(feature_map_class_list)-1)
    print(f"Running with {len(datasets)} datasets, {len(feature_map_class_list)} feature maps, {number_of_investigated_repetition_values} different encoding repetitions for the ZZ feature map, and {cross_validation_splits}-fold cross validation, requiring the training of {combination_count} classifiers in total.")

    # create a list of each possible combination / classifier
    # configuration so that they can be processed in parallel

    ## Process each combination.
    # First, make a list of all classifier configurations (combinations of
    # feature maps, training and testing splits, datasets, repetitions, etc.
    # - all identifying information for each classifier to create)
    print("Generating classifier configurations...")
    combinations = []
    for dataset, dataset_name in zip(datasets, dataset_names):
        # Perform dimensionality reduction and scaling on the features,
        # and transform the labels as done in the qiskit_machine_learning
        # data set loading source code. Also prepares the data for
        # binary rather than multi-class classification if enabled.
        features, labels = process_dataset(dataset, binary_classification=do_binary_classification, qubit_count=qubit_count)
        print(f"{dataset_name} dataset: {len(features)} feature vectors.")
        # For each k-fold split of the data into training and testing sets
        for (split_number, split_tuple) in enumerate(cross_fold_sets(features, labels, k=cross_validation_splits)):
            # For each feature map
            for feature_map_class_number, feature_map_name in zip(feature_map_class_numbers, feature_map_names):
                def record_combination_with_repetitions(repetitions):
                    print(f"Recording combination of data set {dataset_name}, feature map {feature_map_name}, cross-validation split {split_number}, and {repetitions} encoding repetitions...")
                    # Record the combination in the combinations list.
                    process_combination_args = (feature_map_class_number, split_tuple, repetitions, qubit_count)  # to be passed to process_combination
                    identifying_key = (dataset_name, split_number, feature_map_name, repetitions)  # to identify the combination in the results dictionary
                    combinations.append((process_combination_args, identifying_key))
                # For ZZ feature map, try multiple encoding repetitions. For other feature maps,
                # just do 2 repetitions.
                if feature_map_name == "ZZ":
                    for reps in range(2, 2+number_of_investigated_repetition_values):
                        record_combination_with_repetitions(reps)
                else:
                    record_combination_with_repetitions(2)
    # define a function to process the combinations information for a single classifier
    def results_from_combination(combination):
        # unpack the combination
        (pc_args, identifying_key) = combination
        # run experiment for this combination
        results = process_combination(*pc_args)
        # return results using human-readable names and values (other than the classifier instance, which is not human readable)
        return (identifying_key, results)

    # process each combination in parallel
    print("Creating, training, and testing classifiers...")
    key_value_result_pairs = parallel(joblib.delayed(results_from_combination)(combination) for combination in combinations)
    # create the results dictionary
    results_table = {}
    for (k, v) in key_value_result_pairs:
        results_table[k] = v
    return results_table

This cell defines a function that performs the experiments and reports their results.

In [7]:
models = []
def main ():
    results = process_all_combinations()
    for combination in results:
        train, test, model = results[combination]
        models.append(model)
        print(f"Combination {combination} has train/test accuracies {train}/{test}.")
    joblib.dump(models, "trained_models.cache") # save all models to a file so they can be inspected later if desired
    return results

This cell can be evaluated to actually perform the experiments. It can take a few hours to run so loading pre-computed results is preferred.

In [8]:
#results = main()                # uncomment this when you want to run it (to prevent running accidentally)

Running with 4 datasets, 3 feature maps, 4 different encoding repetitions for the ZZ feature map, and 5-fold cross validation, requiring the training of 120 classifiers in total.
Generating classifier configurations...
cancer dataset: 569 feature vectors.
Recording combination of data set cancer, feature map Pauli, cross-validation split 0, and 2 encoding repetitions...
Recording combination of data set cancer, feature map Z, cross-validation split 0, and 2 encoding repetitions...
Recording combination of data set cancer, feature map ZZ, cross-validation split 0, and 2 encoding repetitions...
Recording combination of data set cancer, feature map ZZ, cross-validation split 0, and 3 encoding repetitions...
Recording combination of data set cancer, feature map ZZ, cross-validation split 0, and 4 encoding repetitions...
Recording combination of data set cancer, feature map ZZ, cross-validation split 0, and 5 encoding repetitions...
Recording combination of data set cancer, feature map Paul

Combination ('cancer', 0, 'Pauli', 2) has train/test accuracies 0.9406593406593406/0.8333333333333334.
Combination ('cancer', 0, 'Z', 2) has train/test accuracies 0.9538461538461539/0.956140350877193.
Combination ('cancer', 0, 'ZZ', 2) has train/test accuracies 0.9406593406593406/0.8333333333333334.
Combination ('cancer', 0, 'ZZ', 3) has train/test accuracies 0.8945054945054945/0.7368421052631579.
Combination ('cancer', 0, 'ZZ', 4) has train/test accuracies 0.8637362637362638/0.6929824561403509.
Combination ('cancer', 0, 'ZZ', 5) has train/test accuracies 0.8395604395604396/0.6228070175438597.
Combination ('cancer', 1, 'Pauli', 2) has train/test accuracies 0.9494505494505494/0.8771929824561403.
Combination ('cancer', 1, 'Z', 2) has train/test accuracies 0.945054945054945/0.9824561403508771.
Combination ('cancer', 1, 'ZZ', 2) has train/test accuracies 0.9494505494505494/0.8771929824561403.
Combination ('cancer', 1, 'ZZ', 3) has train/test accuracies 0.8857142857142857/0.7894736842105263

The next 3 cells should be run selectively to save and load pre-computed results. This caching is secondary to the memoization caching on individual functions, it just saves and loads the results variable only.

In [9]:
def save_results():
    """Saves results to a file if there are any, overwriting previous results."""
    if results != None:
        joblib.dump(results, "results.cache")
def load_results():
    """Loads results from a file."""
    global results
    results = joblib.load("results.cache")

In [10]:
#save_results() # uncomment this when you want to run it (to prevent accidentally running it)

In [24]:
#load_results() # uncomment this when you want to run it (to prevent accidentally running it)

These 2 cells read the results variable and combine cross-validation runs to get statistics.

In [11]:
def combine_cross_validations():
    def extract_cross_validations(combination):
        """Returns a list of training and a list of testing accuracies for the given
        combination, using the values in the results from the different cross-validation
        runs."""
        dataset_name, feature_map_name, repetitions = combination
        train_list = []
        test_list = []
        for key in results:
            (d_name, run, f_name, rep) = key
            if d_name == dataset_name and f_name == feature_map_name and rep == repetitions:
                train_accuracy, test_accuracy, model = results[key]
                train_list.append(train_accuracy)
                test_list.append(test_accuracy)
        return train_list, test_list
    # extract combinations to get statistics about from the results variable
    combinations = set()
    for key in results:
        (d_name, run, f_name, rep) = key
        combinations.add((d_name, f_name, rep))
    stats = {}
    for c in combinations:
        train_accuracies, test_accuracies = extract_cross_validations(c)
        stats[c] = ((np.mean(train_accuracies), np.std(train_accuracies), np.var(train_accuracies)),
                    (np.mean(test_accuracies), np.std(test_accuracies), np.var(test_accuracies)))
    return stats

This should only run after computing or loading a value for the results variable.

In [13]:
stats = combine_cross_validations()
# TODO: this below loop should be made more robust to experiment configuration changes if it stays in the code
number_of_investigated_repetition_values = 4
for dataset in ("wine", "cancer", "iris", "digits"):
    for feature_map in ("Pauli", "Z", "ZZ"):
        for repetition in range(2, (2+number_of_investigated_repetition_values) if feature_map == "ZZ" else 3):
            key = (dataset, feature_map, repetition)
            value = stats[key]
            print(f"Stats for {dataset} dataset, {feature_map} feature map, {repetition} repetitions:")
            print("Mean classifier accuracies:")
            print(f"{value[0][0]} | {value[1][0]}")
            print("Standard deviations:")
            print(f"{value[0][1]} | {value[1][1]}")
            print("Variances:")
            print(f"{value[0][2]} | {value[1][2]}")
            print()

Stats for wine dataset, Pauli feature map, 2 repetitions:
Mean classifier accuracies:
0.9673076923076923 | 0.7461538461538462
Standard deviations:
0.014390989949130531 | 0.06249260311258431
Variances:
0.00020710059171597596 | 0.003905325443786982

Stats for wine dataset, Z feature map, 2 repetitions:
Mean classifier accuracies:
0.9442307692307693 | 0.9230769230769231
Standard deviations:
0.01538461538461538 | 0.042132504423474326
Variances:
0.00023668639053254424 | 0.0017751479289940838

Stats for wine dataset, ZZ feature map, 2 repetitions:
Mean classifier accuracies:
0.9673076923076923 | 0.7461538461538462
Standard deviations:
0.014390989949130531 | 0.06249260311258431
Variances:
0.00020710059171597596 | 0.003905325443786982

Stats for wine dataset, ZZ feature map, 3 repetitions:
Mean classifier accuracies:
0.9538461538461538 | 0.5846153846153846
Standard deviations:
0.018644922528524326 | 0.08213137116947161
Variances:
0.000347633136094674 | 0.006745562130177514

Stats for wine data